In [6]:
!pip install pytorchvideo torch torchvision opencv-python

In [29]:
import torch
import cv2
import numpy as np
from torch import nn
from pytorchvideo.models.hub import slowfast_r50
from torchvision.transforms import Compose
import torchvision.transforms._transforms_video as transforms

# Load pretrained SlowFast model and strip classifier head
def load_slowfast_model():
    model = slowfast_r50(pretrained=True)
    model.blocks[-1].proj = nn.Identity()  # remove classification head
    model.eval()
    return model

# Video transform: normalize pixel values
def get_transform():
    return Compose([
        transforms.NormalizeVideo(
            mean=[0.45, 0.45, 0.45],
            std=[0.225, 0.225, 0.225]
        )
    ])

# Load video frames and format for SlowFast
def load_video_frames(path, num_frames=32, slowfast_alpha=4):
    cap = cv2.VideoCapture(path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_rate = max(total_frames // num_frames, 1)

    frames = []
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * sample_rate)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224, 224))
        frames.append(frame)

    cap.release()

    if len(frames) < num_frames:
        raise ValueError(f"Only {len(frames)} frames could be read. Expected {num_frames}.")

    video = np.stack(frames)  # (T, H, W, C)
    video = torch.from_numpy(video).float() / 255.0  # Normalize to [0, 1]
    video = video.permute(3, 0, 1, 2)  # (C, T, H, W)
    video = video.unsqueeze(0)  # (B, C, T, H, W)
    return video

# Prepare video for SlowFast input (two pathways)
def pack_pathway(video, alpha=4):
    fast_pathway = video
    slow_pathway = video[:, :, ::alpha, :, :]
    return [slow_pathway, fast_pathway]

# Main function: extract embedding from video path
def extract_slowfast_embedding(video_path):
    model = load_slowfast_model()
    transform = get_transform()
    video = load_video_frames(video_path)
    video = transform(video.squeeze(0)).unsqueeze(0)
    inputs = pack_pathway(video)

    with torch.no_grad():
        embedding = model(inputs)
    return embedding.squeeze(0)  # shape: (2304,)

# Test path
video_path = "/Users/j/code/bjj_classifier/data/training/pulling_guard/pulling_guard01.mp4"

# Extract and print
embedding = extract_slowfast_embedding(video_path)
print("Embedding shape:", embedding.shape)  # should be torch.Size([2304])
print("Embedding sample:", embedding[:5])    # first 5 values

Embedding shape: torch.Size([2304])
Embedding sample: tensor([0.3879, 0.2860, 0.1396, 0.0334, 0.0007])


In [30]:
name_dict = {"name" : "Jacob"}
name_dict["name"]
os.listdir("../bjj_classifier/data/training/pulling_guard")
!pwd

/Users/j/code/bjj_classifier


In [33]:
import os
data_dict = {}
data_dict["training"] = {}
data_dict["training"]["pulling_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/training/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/training/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["training"]["pulling_guard"][file_name] = embedding

data_dict["training"]["passing_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/training/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/training/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["training"]["passing_guard"][file_name] = embedding
    
data_dict["test"] = {}
data_dict["test"]["pulling_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/test/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/test/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["test"]["pulling_guard"][file_name] = embedding

data_dict["test"]["passing_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/test/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/test/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["test"]["passing_guard"][file_name] = embedding        


        
print(data_dict["test"]["passing_guard"])

{'pulling_guard05.mp4': tensor([0.1640, 0.1205, 0.0652,  ..., 0.0228, 0.4845, 0.0738]), 'pulling_guard06.mp4': tensor([0.0325, 0.1148, 0.0159,  ..., 0.0290, 0.1209, 0.0611])}


In [190]:
import os

def build_dataset(base_path, categories):
    dataset = {} #initialize data set
    for category in categories: #Iterates through training and test
        dataset[category] = {} #Create first keys in data set. First for training, then for test.
        for label in ["pulling_guard", "passing_guard"]: #label gets set to equal both pulling_guard and passing_guard
            label_path = os.path.join(base_path, category, label) #Creates a path based off of label and category
            dataset[category][label] = {} #Creates a second key within the dataset
            for file_name in os.listdir(label_path): #Iterates through all files in dirs with both category and label name
                if not file_name.startswith("."): #Doesn't include .DS_Store
                    video_path = os.path.abspath(label_path + "/" + file_name) #Create path to specific files/video
                    embedding = extract_slowfast_embedding(video_path) #Create embeddings for specific files
                    dataset[category][label][file_name] = embedding #Stores embeddings in data set
    return dataset

data_dict = build_dataset("../bjj_classifier/data", ["training", "test"])

../bjj_classifier/data/training/pulling_guard
../bjj_classifier/data/training/passing_guard
../bjj_classifier/data/test/pulling_guard
../bjj_classifier/data/test/passing_guard


In [ ]:
data_dict

In [ ]:
!pip install scikit-learn

In [55]:
import torch
import torch.nn.functional as F
import sys

embedding_a = data_dict["training"]["pulling_guard"]["pulling_guard02.mp4"]
embedding_b = data_dict["test"]["pulling_guard"]["pulling_guard05.mp4"]
##embedding_b = data_dict["test"]["passing_guard"]["passing_guard05.mp4"]

# Compute cosine similarity between [1, D] vectors
similarity = F.cosine_similarity(embedding_a.unsqueeze(0), embedding_b.unsqueeze(0), dim=1).item()
similarity_percentage = (similarity + 1) / 2 * 100

print(f"Similarity: {similarity_percentage:.2f}%")

Similarity: 87.21%


In [56]:
def perdiction(file_name):
    
    index1 = 0
    
    
    while file_name[index1].isalpha():
        index1 += 1
    

    
    embedding_a = data_dict["test"][file_name[0:len(file_name)-index1+1]][file_name]
    
    embedding_list_pulling = [] #make embedding list for all pulling guard videos
    embedding_list_passing = [] #make embedding list for all passing guard videos

    
    for files in os.listdir("../bjj_classifier/data/training/pulling_guard"): 
        if files != ".DS_Store":
            embedding_list_pulling.append(data_dict["training"]["pulling_guard"][files])

    for files in os.listdir("../bjj_classifier/data/training/passing_guard"): 
        if files != ".DS_Store":
            embedding_list_passing.append(data_dict["training"]["passing_guard"][files])

    
    sum1 = 0
    index2 = 0
    for embeddings in embedding_list_pulling:
        similarity = F.cosine_similarity(embedding_a.unsqueeze(0), embedding_list_pulling[index2].unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        sum1 += similarity_percentage
        index2 += 1

    average_pulling = sum1/len(embedding_list_pulling)
    print(average_pulling)

    sum2 = 0
    index3 = 0
    for embeddings in embedding_list_passing:
        similarity = F.cosine_similarity(embedding_a.unsqueeze(0), embedding_list_passing[index3].unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        sum2 += similarity_percentage
        index3 += 1
        
    average_passing = sum2/len(embedding_list_passing)
    print(average_passing)
    
    if average_passing < average_pulling:
        return "This is pulling guard"
    elif average_passing > average_pulling:
        return "This is passing guard"




In [57]:
perdiction("pulling_guard05.mp4")

81.51419088244438
74.01787005364895


'This is pulling guard'

In [ ]:
dir("hello")

In [291]:

def perdiction02(file_name):
    input_embedding = data_dict["test"][get_dir_name01(file_name)][file_name]
    embedding_list_pulling = making_embedding_list01("pulling_guard")
    embedding_list_passing = making_embedding_list01("passing_guard")
    pulling_sim = compare_embeddings01(input_embedding, embedding_list_pulling)
    passing_sim = compare_embeddings01(input_embedding, embedding_list_passing)
    print(pulling_sim)
    print(passing_sim)
    return compare01(pulling_sim, passing_sim)

def get_dir_name01(file_name):
    index = 0
    
    while file_name[index].isalpha():
        index += 1
    
    return file_name[0:len(file_name)-index+1]


def making_embedding_list01(dir):
    embedding_list = []
    for files in os.listdir(f"../bjj_classifier/data/training/{dir}"): 
        if files != ".DS_Store":
            embedding_list.append(data_dict["training"][dir][files])

    return embedding_list


def compare_embeddings01(input_embedding, embedding_list):
    sum = 0
    index = 0
    for embeddings in embedding_list:
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), embeddings.unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        sum += similarity_percentage
        index += 1

    average_sim = sum/len(embedding_list)
    return average_sim

def compare01(pulling_sim, passing_sim):
    if passing_sim < pulling_sim:
        return "This is pulling guard"
    elif passing_sim > pulling_sim:
        return "This is passing guard"


In [292]:
print(perdiction02("passing_guard05.mp4"))

77.81077697873116
77.59945914149284
This is pulling guard


In [385]:
import string
def perdiction03(file_name):
    input_embedding = data_dict["test"][get_dir_name(file_name)][file_name]
    dir_dict = {}
    for dir in os.listdir("../bjj_classifier/data/training"):
        if dir != ".DS_Store":
            embedding_list = making_embedding_list02(dir)
            dir_dict[dir] = embedding_list
            
    
    
    embedding_sim_dict = {}
    for dir in os.listdir("../bjj_classifier/data/training"):
       if dir != ".DS_Store":
           embedding_sim_dict[dir] = compare_embeddings02(input_embedding, dir_dict[dir])

    print(embedding_sim_dict["pulling_guard"])
    print(embedding_sim_dict["passing_guard"])
    
    
    return compare02(embedding_sim_dict)
    

def get_dir_name02(file_name):
    index = 0
    
    while file_name[index].isalpha():
        index += 1
    
    return file_name[0:len(file_name)-index+1]


def making_embedding_list02(dir):
    embedding_list = []
    for files in os.listdir(f"../bjj_classifier/data/training/{dir}"): 
        if files != ".DS_Store":
            embedding_list.append(data_dict["training"][dir][files])

    return embedding_list


def compare_embeddings02(input_embedding, embedding_list):
    sum = 0
    index = 0
    for embeddings in embedding_list:
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), embeddings.unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        sum += similarity_percentage
        index += 1

    average_sim = sum/len(embedding_list)
    return average_sim

def compare02(embedding_sim_dict):
    final_string = ""
    percent = 0
    for dir in embedding_sim_dict:
        if embedding_sim_dict[dir] > percent:
            percent = embedding_sim_dict[dir]
            final_string += (str(dir))
            final_string = ''.join([c if c not in string.punctuation else ' ' for c in final_string])

    return f"This is {final_string}"
            



In [386]:
print(perdiction03("passing_guard06.mp4"))

76.9784677773714
79.95940744876862
This is passing guard


In [295]:
def perdiction04(file_name):
    input_embedding = data_dict["test"][get_dir_name03(file_name)][file_name]
    embedding_list_pulling = making_embedding_list03("pulling_guard")
    embedding_list_passing = making_embedding_list03("passing_guard")
    pulling_sim = compare_embeddings03(input_embedding, embedding_list_pulling)
    passing_sim = compare_embeddings03(input_embedding, embedding_list_passing)
    print(pulling_sim)
    print(passing_sim)
    return compare03(pulling_sim, passing_sim)

def get_dir_name03(file_name):
    index = 0
    
    while file_name[index].isalpha():
        index += 1
    
    return file_name[0:len(file_name)-index+1]


def making_embedding_list03(dir):
    embedding_list = []
    for files in os.listdir(f"../bjj_classifier/data/training/{dir}"): 
        if files != ".DS_Store":
            embedding_list.append(data_dict["training"][dir][files])

    return embedding_list


def compare_embeddings03(input_embedding, embedding_list):
    compare = 0
    index = 0
    for embeddings in embedding_list:
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), embeddings.unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        if similarity_percentage > compare:
            compare = similarity_percentage

    return compare

def compare03(pulling_sim, passing_sim):
    if passing_sim < pulling_sim:
        return "This is pulling guard"
    elif passing_sim > pulling_sim:
        return "This is passing guard"




In [296]:
print(perdiction04("passing_guard05.mp4"))

81.21072351932526
82.1433424949646
This is passing guard
